In [4]:
import pandas as pd
from string import punctuation
import re
import numpy as np
from setup import *
import glob
import os
import pickle
import joblib
from collections import Counter
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize, TweetTokenizer
from nltk.corpus import stopwords
from matplotlib import pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

keep_reach = 50


In [3]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

# The Process

## [I. Data Acquisition](#data_acquisition_link)

## [II. Preprocessing](#preprocessing_link)

## [III. The Model](#model_link)
1. [Grid Search](#grid_search_link)
1. [Cross Validate](#cv_link)
1. [Save the best with Pickle](#pickle_link)


<a id='data_acquisition_link'></a>
# Data Acquisition
1. Check if Files have been collected already
1. Use Glob to collect all of the files in the Data Folder
1. Save to single file for later use

In [5]:

if os.path.isfile('All_Files.csv'):
    print('Using old dataframe')
    df = pd.read_csv('All_Files.csv')

else:
    print('Making new dataframe')
    file_list = glob.glob("./data/Facebook Insights*")

    df_list = []

    for file in file_list:
        df_list.append(pd.read_csv(file,encoding='iso-8859-1').loc[1:])

    df = pd.concat(df_list).reset_index()
    df.to_csv('All_Files.csv')
display(df)

Using old dataframe


Unnamed: 0  index                           Post ID  \
0              0      1   171162119680585_859052127558244   
1              1      2   171162119680585_859037424226381   
2              2      3   171162119680585_859081790888611   
3              3      4   171162119680585_859035847559872   
4              4      5   171162119680585_859091770887613   
5              5      6   171162119680585_859074730889317   
6              6      7   171162119680585_859035654226558   
7              7      8   171162119680585_859058130890977   
8              8      9   171162119680585_859028140893976   
9              9     10   171162119680585_859018744228249   
10            10     11   171162119680585_857911724338951   
11            11     12   171162119680585_855331344596989   
12            12     13   171162119680585_855328057930651   
13            13     14   171162119680585_855323207931136   
14            14     15  171162119680585_1753599848222100   
15            15     16   171162119680585_856986864431437   
16            16     17   171162119680585_855306601266130   
17            17     18   171162119680585_855304154599708   
18            18     19   171162119680585_855301137933343   
19            19     20   171162119680585_855286651268125   
20            20     21   171162119680585_855283257935131   
21            21     22   171162119680585_855276011269189   
22            22     23   171162119680585_855265557936901   
23            23     24   171162119680585_851535571643233   
24            24     25   171162119680585_849300141866776   
25            25     26   171162119680585_853674101429380   
26            26     27   171162119680585_851533761643414   
27            27     28   171162119680585_851531261643664   
28            28     29   171162119680585_849298855200238   
29            29     30   171162119680585_851530084977115   
...          ...    ...                               ...   
1023        1023     33  171162119680585_1162108553919265   
1024        1024     34  171162119680585_1159616344168486   
1025        1025     35  171162119680585_1162056530591134   
1026        1026     36  171162119680585_1161476073982513   
1027        1027     37  171162119680585_1159611647502289   
1028        1028     38  171162119680585_1159608070835980   
1029        1029     39  171162119680585_1159597077503746   
1030        1030     40  171162119680585_1159591824170938   
1031        1031     41  171162119680585_1155131721283615   
1032        1032     42  171162119680585_1155143631282424   
1033        1033     43  171162119680585_1155118177951636   
1034        1034     44  171162119680585_1155111614618959   
1035        1035     45  171162119680585_1155104857952968   
1036        1036     46  171162119680585_1153349054795215   
1037        1037     47  171162119680585_1151876408275813   
1038        1038     48  171162119680585_1151846311612156   
1039        1039     49  171162119680585_1151840804946040   
1040        1040     50  171162119680585_1151164051680382   
1041        1041     51  171162119680585_1146796018783852   
1042        1042     52  171162119680585_1146783035451817   
1043        1043     53  171162119680585_1146755258787928   
1044        1044     54  171162119680585_1147883188675135   
1045        1045     55  171162119680585_1146744692122318   
1046        1046     56  171162119680585_1146740852122702   
1047        1047     57  171162119680585_1146748288788625   
1048        1048     58  171162119680585_1146745715455549   
1049        1049     59  171162119680585_1146730522123735   
1050        1050     60  171162119680585_1145064398957014   
1051        1051     61  171162119680585_1144567672340020   
1052        1052     62  171162119680585_1144038895726231   

                                              Permalink  \
0     https://www.facebook.com/TarletonCOGS/posts/85...   
1     https://www.facebook.com/TarletonCOGS/posts/85...   
2     https://www.facebook.com/TarletonCOGS/po

<a id='preprocessing_link'></a>
# Preprocessing

1. Collect the columns for reach, engagement
1. Dropna, since they will not contribute to finding good posts
1. Create proportion column
1. Tokenize
1. Tf-Idf
1. Save the Tf-Idf using pickle
1. Condense the preprocessing step into a function ** TO DO STILL **


Use This next cell to restart the fresh df, to check changes in the preprocessing

In [8]:
overwrite = True

if os.path.isfile('vectorizer.pkl') and overwrite is False:
    print('Using old tokenized/scaled')
    df = pd.read_csv('All_Files.csv')
    X_all = pd.read_csv('all_X.csv',header=None, index_col = 0)[1]
    y_all = pd.read_csv('all_y.csv',header=None, index_col = 0,skiprows = 1)[1]
else:
    print('Creating new tokenized/scaled')
    df = pd.read_csv('All_Files.csv')
    df = df[['Post Message', 'Type', 'Posted', 'Lifetime Post Total Reach','Lifetime Engaged Users']]
    df.columns = ['Post', 'Type','Date','Reach','Engaged']
    df.dropna(inplace=True)
    df = pd.merge(df.groupby('Post')['Engaged'].apply(lambda p: max(p)).reset_index(),df[['Post', 'Type', 'Date', 'Reach', 'Engaged']], on = ['Post','Engaged'], how = 'inner')
    df.drop_duplicates('Post',inplace=True)
    df = df.loc[df['Reach']>=keep_reach].reset_index(drop=True)
    df['Prop'] = df['Engaged']/df['Reach']
    tokenized_df = df
    tokenized_df['Sentences'] = tokenized_df['Post'].transform(lambda x: sent_tokenize(str(x)))
    tokenized_df['Words'] = tokenized_df['Post'].transform(lambda x: ' '.join([WordNetLemmatizer().lemmatize(y.lower()) for y in word_tokenize(x) if y.isalpha()]))
    tokenized_df['Words'] = tokenized_df['Words'].transform(lambda x: ' '.join([y for y in word_tokenize(x) if y not in stopwords.words('english')]))
    tokenized_df = tokenized_df.loc[tokenized_df['Words'] != 'http']
    X_all = tokenized_df['Words']
    scaler = MinMaxScaler()
    scaler.fit(tokenized_df['Prop'].values.reshape(-1,1))
    y_all = scaler.transform(tokenized_df['Prop'].values.reshape(-1,1)).ravel()
    joblib.dump(scaler, 'minmax.pkl')
    X_all.to_csv('all_X.csv')
    pd.DataFrame(y_all).to_csv('all_y.csv') 

print('Done')


Creating new tokenized/scaled
Done


In [12]:
y_all.shape
X_all.shape

(779,)

<a id='model_link'></a>
<a id='grid_search_link'></a>
## Grid Search
1. First we need to setup a pipeline to Vectorize the posts (but only the training set)
1. Setup the grid search to run with pipeline
1. Find the best model from the grid search

<a id='grid_search_link'></a>
### Grid Search
We want to grid search over the tunable parameters of the SVR model.<br>
__Parameters__:<br>
__kernel__: linear, poly, rbf, sigmoid, or callable. This defines how the split will occur.<br>
__degree__: if polynomial then it is the degree of the polynomial for the split.<br>
__gamma__: kernel coefficient for rbf, poly and sigmoid. must be float or auto. If auto, then 1/n features will be used.<br>
__coef0__: The independent term of the kernel function. Only used in poly and sigmoid.<br>

In [28]:
sig_pipe = Pipeline([('tfidf',TfidfVectorizer()),('svr', SVR(kernel = 'sigmoid'))])
sig_parameters = {'tfidf__min_df':(0.01,0.02,0.05,0.1,0.15),'tfidf__max_df':(0.4, 0.5,0.65,0.85,1.0),'tfidf__ngram_range':((1,1),(1,2),(1,3),(1,4)),\
              'tfidf__max_features': (None, 50, 100,250),'svr__C':(1,10,100,1000), 'svr__gamma': (0.001,0.0001), 'svr__coef0':(0.0,0.1,0.25,0.5,1.0,2.0,4.0)}


sig_grid = GridSearchCV(sig_pipe, sig_parameters, verbose = 1, cv=5, n_jobs = -1)
print(X_all.shape, y_all.shape)
sig_grid.fit(X_all, y_all)
print(sig_grid.best_score_)
print(sig_grid.best_params_)

(779,) (779,)
Fitting 5 folds for each of 22400 candidates, totalling 112000 fits


[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 1024 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 2524 tasks      | elapsed:   28.7s
[Parallel(n_jobs=-1)]: Done 4624 tasks      | elapsed:   54.1s
[Parallel(n_jobs=-1)]: Done 7324 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 10624 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 14524 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 19024 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 24124 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 29824 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 36124 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 43024 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done 50524 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-1)]: Done 58624 tasks      | elapsed: 11.8min
[Parallel(n_jobs=-1)]: Done 67324 tasks      | elapsed: 13.6min
[Parallel(n_jobs=-1)]: Done 76624 tasks      |

0.09559188188778658
{'svr__C': 1000, 'svr__coef0': 2.0, 'svr__gamma': 0.001, 'tfidf__max_df': 0.4, 'tfidf__max_features': None, 'tfidf__min_df': 0.02, 'tfidf__ngram_range': (1, 1)}


In [ ]:

poly_pipe = Pipeline([('tfidf',TfidfVectorizer()),('svr', SVR(kernel = 'poly'))])
poly_parameters = {'tfidf__min_df':(0.01,0.02,0.05,0.1,0.15),'tfidf__max_df':(0.4, 0.5,0.65,0.85,1.0),'tfidf__ngram_range':((1,1),(1,2),(1,3),(1,4)),\
              'tfidf__max_features': (None, 50, 100,250),'svr__coef0':(0.0,0.1,0.25,0.5,1.0,2.0,5.0), 'svr__C':(1,10,100,1000), 'svr__gamma': (0.001,0.0001), 'svr__degree':(3,4,5)}


poly_grid = GridSearchCV(poly_pipe, poly_parameters, verbose = 1, cv=5, n_jobs = -1)
poly_grid.fit(X_all, y_all)
print(poly_grid.best_score_)
print(poly_grid.best_params_)

Fitting 5 folds for each of 67200 candidates, totalling 336000 fits


[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 1024 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done 2524 tasks      | elapsed:   27.3s
[Parallel(n_jobs=-1)]: Done 4624 tasks      | elapsed:   50.0s
[Parallel(n_jobs=-1)]: Done 7324 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 10624 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 14524 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 19024 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 24124 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 29824 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 36124 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 43024 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 50524 tasks      | elapsed:  9.8min
[Parallel(n_jobs=-1)]: Done 58624 tasks      | elapsed: 11.4min
[Parallel(n_jobs=-1)]: Done 67324 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done 76624 tasks      |

In [29]:
rbf_pipe = Pipeline([('tfidf',TfidfVectorizer()),('svr', SVR(kernel = 'rbf'))])
rbf_parameters = {'tfidf__min_df':(0.01,0.02,0.05,0.1,0.15),'tfidf__max_df':(0.4, 0.5,0.65,0.85,1.0),'tfidf__ngram_range':((1,1),(1,2),(1,3),(1,4)),\
              'tfidf__max_features': (None, 50, 100,250),'svr__C':(1,10,100,1000),'svr__gamma': (0.001,0.0001)}

rbf_grid = GridSearchCV(rbf_pipe, rbf_parameters, verbose = 1, cv=5, n_jobs = -1)
rbf_grid.fit(X_all, y_all)
print(rbf_grid.best_score_)
print(rbf_grid.best_params_)

Fitting 5 folds for each of 3200 candidates, totalling 16000 fits


[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 1024 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-1)]: Done 2524 tasks      | elapsed:   29.8s
[Parallel(n_jobs=-1)]: Done 4624 tasks      | elapsed:   56.2s
[Parallel(n_jobs=-1)]: Done 7324 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 10624 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 13852 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 15985 out of 16000 | elapsed:  3.4min remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 16000 out of 16000 | elapsed:  3.4min finished


0.08899502068141238
{'svr__C': 1000, 'svr__gamma': 0.0001, 'tfidf__max_df': 0.4, 'tfidf__max_features': 100, 'tfidf__min_df': 0.02, 'tfidf__ngram_range': (1, 3)}


In [184]:
from sklearn.metrics import mean_absolute_error
model= sig_grid.best_estimator_

best_of_best = []
models = [sig_grid.best_estimator_,poly_grid.best_estimator_,rbf_grid.best_estimator_]


depth = 2000

for x in models:
    best = 1
    run = 0
    for y in range(depth):
        X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.3)
        x.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        run+=mean_absolute_error(y_pred,y_test)
        if mean_absolute_error(y_pred,y_test) < best:
            best = mean_absolute_error(y_pred,y_test)
    best_of_best.append([run/depth,best])


for x in best_of_best:
    print(x[0],x[1])




0.08233926369898034 0.07036004506226383
0.07942311527147757 0.06754033707813559
0.07922323968385168 0.06541807283992555
